## Short comment

In this we continue Lab 2, however this is based on lecture notes only. I am comparing two different machine learning approaches - Bagging with Decision Tree and FNN with dropout- on the Forest CoverType from sklearn dataset (581 012 rows, 54 columns, and 7 classes to predict).
I am not optimizing either machine learning approaches (normalization  prior the FNN, Cross-validation etc etc), the intention was just to apply theory into practice on these new approaches. 
The result was not surprising.

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [4]:
# Defining the dataset from sklearn, returns like a Pandas Dataframe
data = datasets.fetch_covtype(as_frame=True)

# Defining the X's and Y's
x = data["data"]
y = data["target"]-1

# Splitting the data into training and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state=42)

In [8]:
print(data ,2)

{'data':         Elevation  Aspect  Slope  Horizontal_Distance_To_Hydrology  \
0          2596.0    51.0    3.0                             258.0   
1          2590.0    56.0    2.0                             212.0   
2          2804.0   139.0    9.0                             268.0   
3          2785.0   155.0   18.0                             242.0   
4          2595.0    45.0    2.0                             153.0   
...           ...     ...    ...                               ...   
581007     2396.0   153.0   20.0                              85.0   
581008     2391.0   152.0   19.0                              67.0   
581009     2386.0   159.0   17.0                              60.0   
581010     2384.0   170.0   15.0                              60.0   
581011     2383.0   165.0   13.0                              60.0   

        Vertical_Distance_To_Hydrology  Horizontal_Distance_To_Roadways  \
0                                  0.0                            510.0   


## Bagging with Decision Trees

In [10]:
# Initilize Decision Tree Classifier as the base model
model_base = DecisionTreeClassifier()

# Initialize Bagging (Bootstrap Aggregation) with 50 base estimators in the ensemble and using all processors (n_jobs=-1)
bagging_model = BaggingClassifier(estimator = model_base, n_estimators=50, random_state=42, n_jobs=-1)

# Train the Bagging model on the training data
bagging_model.fit(x_train, y_train)

# Prediction on the test data
y_pred_bagging = bagging_model.predict(x_test)

# Calculate accuracy for Bagging
acc_bagging = accuracy_score(y_test, y_pred_bagging)

Bagging Test Accuracy: 0.9667


## Sequential Neural Network with Dropout

In [21]:
# Defining the NN as a sequential model
hidden_layer_size = 128
nn_model = models.Sequential([
    # Input layer
    layers.Input(shape = (x_train.shape[1],)),
    # First hidden layer
    layers.Dense(hidden_layer_size, activation="relu"),
    # Adding Dropout of 50%
    layers.Dropout(0.5),
    # Hidden Layer
    layers.Dense(hidden_layer_size, activation="relu"),
    # Adding Dropout of 50%
    layers.Dropout(0.5),
    # Output layer for classification (7 classes in the dataset)
    layers.Dense(7, activation="softmax")
])

# Summary of the model
nn_model.summary()

# Compile the model
nn_model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Train the model with Dropout
nn_model_trained = nn_model.fit(x_train, y_train, epochs=50, batch_size=128, validation_data=(x_test, y_test), verbose=0)

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_23 (Dense)                │ (None, 128)            │         7,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_16 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_17 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 7)              │           903 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,455 (95.53 KB)

 Trainable params: 24,455 (95.53 KB)

 Non-trainable params: 0 (0.00 B)

In [23]:
# Evaluate NN and Bagging
test_loss_nn, test_acc_nn = nn_model.evaluate(x_test, y_test, verbose=0)
print(f"Neural Network with Dropout Test Accuracy: {test_acc_nn:.4f}")

# Printing out the accuracy for Bagging
print(f"Bagging Test Accuracy: {acc_bagging:.4}")

Neural Network with Dropout Test Accuracy: 0.6798
Bagging Test Accuracy: 0.9667


Typically, Bagging performs well on structured/tabular data like the Forest CoverType dataset, and we see that by the high accuracy of 96.67%. This is because Bagging reduces variance by combining the predictions of multiple decision trees trained on different subsets of the data.

The neural network performed lower in accuracy compared to Bagging, especially since it's not fully tuned for this kind of structured data. While Dropout helps prevent overfitting, neural networks usually require more fine-tuning for tabular datasets, its accuracy is significantly lower of 67.98%.